In [ ]:
import pandas as pd
import plotly.graph_objects as go
import plotly.express as px
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings("ignore")
plt.style.use('ggplot')

In [ ]:
data = pd.read_csv('combined_file.csv')

In [ ]:
data.Name.unique()

In [ ]:
data.drop(data.query('Name == "Name"').index, inplace=True)

In [ ]:
data.isna().sum()

In [ ]:
data.info()

In [ ]:
data['Date'] = pd.to_datetime(data['Date'])

In [ ]:
for x in data.columns:
    if data[x].dtype == 'O' and x != 'Name':
        data[x] = data[x].astype(float)

In [ ]:
data.info()

In [ ]:
df = data.Name.value_counts().reset_index()

fig, ax = plt.subplots()
ax.pie(
    x=df['Name'],
    startangle=90,
    labels=df['index'],
    autopct="%.0f%%",
#     shadow=True,
#         explode=[0.1, 0]
            )

plt.title('Brand Distributio in Percentage');

In [ ]:
from dash import Dash, html, dcc
from jupyter_dash import JupyterDash
from dash.dependencies import Output, Input
from dash.exceptions import PreventUpdate

In [ ]:
data

In [ ]:
#Stock Selection based on Company Name with Dropdown

In [ ]:
app = JupyterDash(__name__)

app.layout = html.Div([
    html.H3('Stock Selection:'),
    dcc.Dropdown(
        id='dropdown',
        options=data.Name.unique(),
        value='AMZN',
    ),
    dcc.Graph(id='visual')
])

@app.callback(Output('visual', 'figure'), Input('dropdown', 'value'))
def stocl_brand(company):
    fig = px.line(
        data.loc[data['Name'] == company],
        x='Date',
        y='Open',
        title=f'{company} Open Price July 2018 to July 2023'
    ).update_layout(
        title={
        'x':0.5,
        'y':0.85,
        'xanchor':'center'}
    ).update_yaxes(tickprefix="$")
    
    return fig

if __name__ == '__main__':
    app.run_server(mode='inline', debug=True, port=8056)

In [ ]:
#Stock Selection based on Company Name with Multi Selection 

In [ ]:
app = JupyterDash(__name__)

app.layout = html.Div([
    html.H3('Stock Selection:'),
    dcc.Dropdown(
        id='dropdown',
        options=data.Name.unique(),
        value=['AMZN', 'GOOG'],
        multi=True 
    ),
    dcc.Graph(id='visual')
])

@app.callback(Output('visual', 'figure'), Input('dropdown', 'value'))
def stocl_brand(company):
    fig = px.line(
        data.query("Name == @company"),
        x='Date',
        y='Open',
        color='Name',
        title=f'{company} Open Price July 2018 to July 2023'
    )
    
    fig.update_layout(
        title={
        'x':0.5,
        'y':0.85,
        'xanchor':'center'}
    ).update_yaxes(tickprefix="$")
    
    return fig

if __name__ == '__main__':
    app.run_server(mode='inline', debug=True, port=8057)

In [ ]:
app = JupyterDash(__name__)

app.layout = html.Div([
    html.H3('Stock Selection:'),
    dcc.Dropdown(
        id='dropdown',
        options=data.Name.unique(),
        value=['AMZN', 'GOOG'],
        multi=True 
    ),
    dcc.Graph(id='visual1'),
    dcc.Graph(id='visual2')
])

@app.callback([Output('visual1', 'figure'),
               Output('visual2', 'figure')], Input('dropdown', 'value'))

def stocl_brand(company):
    
    df = data.query("Name == @company")
    
    fig1 = px.line(
        df,
        x='Date',
        y='Open',
        color='Name',
        title=f'{company} Open Price July 2018 to July 2023'
    )
    
    fig1.update_layout(
        title={
        'x':0.5,
        'y':0.85,
        'xanchor':'center'}
    ).update_yaxes(tickprefix="$")
    
    fig2 = px.bar(
        df.groupby('Name', as_index=False).sum().sort_values('Volume',ascending=False),
        x='Volume',
        y='Name',
        color='Name',
        labels={'Name':'Company'},
        title = f'{company} Total Volume'
    ).update_layout(height=250).update_layout(
        title={
        'x':0.5,
        'y':0.85,
        'xanchor':'center'}
    ).update_xaxes(tickprefix="$")
    
    return fig1, fig2

if __name__ == '__main__':
    app.run_server(mode='inline', debug=True, port=8058)

In [ ]:
#Stock Selection in different year of different company with Multi Selection 

In [ ]:
data['Year'] = data['Date'].dt.year

In [ ]:
data.head()

In [ ]:
app = JupyterDash(__name__)

app.layout = html.Div([
    html.H3('Stock Selection:'),
    dcc.Dropdown(
        id='dropdown',
        options=data.Year.unique(),
        value = 2018
    ),
    dcc.Graph(id='visual1'),
    dcc.Graph(id='visual2')
    
])

@app.callback([Output('visual1', 'figure'),
               Output('visual2', 'figure')], Input('dropdown', 'value'))
def stocl_brand(year):
    df = data.loc[data['Year'] == year]
    
    fig1 = px.line(
        df,
        x='Date',
        y='Open',
        color='Name',
        title=f'Company Open Price in {year}'
    )
    
    fig1.update_layout(
        title={
        'x':0.5,
        'y':0.85,
        'xanchor':'center'}
    ).update_yaxes(tickprefix="$")
    
    
    fig2 = px.bar(
        df.groupby(['Name', 'Year'], as_index=False).sum().sort_values('Volume', ascending=False),
        x='Volume',
        y='Name',
        color='Name',
        labels={'Name':'Company'}
    )
    
    return fig1, fig2

if __name__ == '__main__':
    app.run_server(mode='inline', debug=True, port=8059)

In [ ]:
#Date Selection with Company Selection 

In [ ]:
app = JupyterDash(__name__)

app.layout = html.Div([
    dcc.DatePickerRange(
        id='picker',
        min_date_allowed=data.Date.min(), #date(2022, 1, 2)
        max_date_allowed=data.Date.max(),
        initial_visible_month=data.Date.min(),
        start_date=data.Date.min(),
        end_date=data.Date.max(),
        display_format='YYYY-MM-DD'
    ),
    dcc.Graph(id='visual1'),
    dcc.Graph(id='visual2')
])

@app.callback([Output('visual1', 'figure'),
              Output('visual2', 'figure')], 
              
              [Input('picker','start_date'),
              Input('picker','end_date')])

def date_plot(start_date, end_date):
    df = data.loc[data['Date'].between(start_date, end_date)]
    
    fig1 = px.line(
        df,
        x='Date',
        y='Open',
        color='Name',
        title=f'Company Stock Open Price between {start_date[:10]} and {end_date[:10]}',
    ).update_layout(
        title={
        'x':0.5,
        'y':0.85,
        'xanchor':'center'}
    ).update_yaxes(tickprefix="$")
    
    fig1.layout.legend.title = 'Company'
    
    fig2 = px.bar(
        df.groupby('Name', as_index=False).sum().sort_values('Volume', ascending=False),
        x='Volume',
        y='Name',
        color='Name',
        title=f'Company Stock Total Volume between {start_date[:10]} and {end_date[:10]}'
    ).update_layout(
        title={
        'x':0.5,
        'y':0.85,
        'xanchor':'center'}
    ).update_xaxes(tickprefix="$")
    
    fig2.layout.legend.title = 'Company'
    
    return fig1, fig2
    
if __name__ == '__main__':
    app.run_server(mode='inline', debug=True, port=8061)

In [ ]:
app = JupyterDash(__name__)

app.layout = html.Div([
    html.H3('Stock Selection:'),
    dcc.Dropdown(
        id='dropdown',
        options=data.Name.unique(),
        value='AMZN',
        multi=True
    ),
    html.H3('Date Selection:'),
    dcc.DatePickerRange(
        id='picker',
        min_date_allowed=data.Date.min(), #date(2022, 1, 2)
        max_date_allowed=data.Date.max(),
        initial_visible_month=data.Date.min(),
        start_date=data.Date.min(),
        end_date=data.Date.max(),
        display_format='YYYY-MM-DD'
    ),
    dcc.Graph(id='visual1'),
    dcc.Graph(id='visual2')
])

@app.callback([Output('visual1', 'figure'),
              Output('visual2', 'figure')], 
              
              [Input('dropdown','value'),
              Input('picker','start_date'),
              Input('picker','end_date')])

def date_plot(company, start_date, end_date):
    df = data.query("Name == @company")
    dfm = df.loc[df['Date'].between(start_date, end_date)]
    
    fig1 = px.line(
        dfm,
        x='Date',
        y='Open',
        color='Name',
        title=f'Company Stock Open Price between {start_date[:10]} and {end_date[:10]}',
    ).update_layout(
        title={
        'x':0.5,
        'y':0.85,
        'xanchor':'center'}
    ).update_yaxes(tickprefix="$")
    
    fig1.layout.legend.title = 'Company'
    
    fig2 = px.bar(
        dfm.groupby('Name', as_index=False).sum().sort_values('Volume', ascending=False),
        x='Volume',
        y='Name',
        color='Name',
        labels={'Name':'Company'},
        title=f'Company Stock Total Volume between {start_date[:10]} and {end_date[:10]}'
    ).update_layout(
        title={
        'x':0.5,
        'y':0.85,
        'xanchor':'center'}
    ).update_xaxes(tickprefix="$")
    
    fig2.layout.legend.title = 'Company'
    
    return fig1, fig2
    
if __name__ == '__main__':
    app.run_server(mode='inline', debug=True, port=8061)

In [ ]:
#sum, mean, sum, for Open price in each stock 

In [ ]:
app = JupyterDash(__name__)

app.layout = html.Div([
    html.H3('Metric Selection:'),
    dcc.RadioItems(
        id='items',
        options=['mean', 'median', 'sum'],
        value='mean'
    ),
    dcc.Graph(id='visual')
])

@app.callback(Output('visual', 'figure'), Input('items', 'value'))
def stock_plot(metric):
    df=data.groupby('Name', as_index=False).agg({'Open':metric})
    fig = px.bar(
        df.sort_values('Open', ascending=False),
        x='Open',
        y='Name',
        color='Name',
        labels={'Name':'Company'},
        title=f'{metric.title()} of Open Price of Different Companies'
    ).update_xaxes(tickprefix="$").update_layout(
        title={
        'x':0.5,
        'y':0.85,
        'xanchor':'center'})
    
    return fig
    
if __name__ == '__main__':
    app.run_server(mode='inline', debug=True, port=8060)

In [ ]:
#Interactive Correlation plot

In [ ]:
data.query('Name=="AAPL"').corr()

In [ ]:
app = JupyterDash(__name__)

app.layout = html.Div([
    html.H3('Company Selection:'),
    dcc.RadioItems(
        id='items1',
        options=data.Name.unique(),
        value='AMZN'
    ),
    html.H3('Variable Selection:'),
    dcc.Dropdown(
        id='x_dropdown',
        options=data.corr().columns,
        value='Open'
    ),
    dcc.Dropdown(
        id='y_dropdown',
        options=data.corr().columns,
        value='High'
    ),
    html.H3('Trendline Option:'),
    dcc.RadioItems(
        id='items2',
        options=['Trendline On', 'Trendline Off'],
        value='Trendline Off'
    ),
    dcc.Graph(id='visual'),
])

@app.callback(
    Output('visual', 'figure'),
    [Input('items1', 'value'),
     Input('items2', 'value'),
     Input('x_dropdown', 'value'),
     Input('y_dropdown', 'value')]
)

def plot(company, switch, x, y):
    df = data.loc[data['Name'] == company]

    fig = px.scatter(
        df,
        x=x,
        y=y,
        trendline= None if switch == 'Trendline Off' else 'ols',
        title = (f'{company} - {x} vs. {y}')
    ).update_xaxes(tickprefix="$")\
    .update_layout(
        title={
        'x':0.5,
        'y':0.85,
        'xanchor':'center'})\
    .update_yaxes(tickprefix="$")

    return fig
    
if __name__ == '__main__':
    app.run_server(mode='inline', debug=True, port=8060)